HuuggingFace

In [1]:
from transformers import __version__

print(f'Transformers version: {__version__}')

Transformers version: 4.35.2


pip install torch -U

pip install transformers -U

pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

pip install accelerate

pip install bitsandbytes

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, os
# from accelerate import Accelerator


In [51]:
torch.cuda.empty_cache()

In [54]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  12916 MiB |  12916 MiB |  12916 MiB |  68608 B   |
|       from large pool |  12852 MiB |  12852 MiB |  12852 MiB |      0 B   |
|       from small pool |     64 MiB |     64 MiB |     64 MiB |  68608 B   |
|---------------------------------------------------------------------------|
| Active memory         |  12916 MiB |  12916 MiB |  12916 MiB |  68608 B   |
|       from large pool |  12852 MiB |  12852 MiB |  12852 MiB |

In [53]:
mn = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0)#, load_in_8bit=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [55]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "who are you?"
toks = tokr(prompt, return_tensors="pt")


In [56]:
toks = toks.to("cpu")

In [57]:
toks

{'input_ids': tensor([[    1,  1058,   526,   366, 29973]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [58]:
tokr.batch_decode(toks['input_ids'])

['<s> who are you?']

In [59]:
res = model.generate(**toks.to("cuda"), max_new_tokens=50).to('cpu')
res

tensor([[    1,  1058,   526,   366, 29973,    13,    13, 29902,   626,   263,
         29871, 29941, 29900, 29899,  6360, 29899,  1025,  6114,   515,   278,
          3303,  3900, 29889,   306,   505,  1063,  8471,   297,  5546,   363,
           278,  4940, 29871, 29945,  2440, 29892,   322,   306,   626,  5279,
          1985,   408,   385,  4223, 15703,   472,   263,   970,  1880,  3762,
         29889,   306, 13389,  5183, 29892]])

In [60]:
text = tokr.batch_decode(res)

In [61]:
print(text[0])

<s> who are you?

I am a 30-year-old woman from the United States. I have been living in Japan for the past 5 years, and I am currently working as an English teacher at a public high school. I enjoy reading,


In [8]:
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)